In [ ]:

import pandas as pd

# Read data from the first table
data = pd.read_excel('/content/test_data.xlsx')

# Create sets of unique operations, rooms, and departments
operations = data['Operation ID'].unique().tolist()
rooms = data['Operating Room'].unique().tolist()
departments = data['Department'].unique().tolist()
days = range(1, 10)

# Create dictionaries for operation times and anesthesia
operation_times = dict(zip(operations, data['Operation Time']))
anesthesia_times = dict(zip(operations, data['Anesthesia']))

# Read data from the second table (weight matrix)
weights_data = pd.read_excel('/content/book_test.xlsx', index_col=0, header=0)

# Create a dictionary of weights
weights = {}
for operation in operations:
    weights[operation] = {room: weights_data.loc[operation, str(room)] for room in rooms}

# Create a dictionary for room daily times
room_daily_times = {room: {day: 720 for day in days} for room in rooms}

def heuristic_algorithm(operations, operation_times, weights, rooms, days, room_daily_times):
    # Initialize schedule and available time
    schedule = {room: {day: [] for day in days} for room in rooms}
    waiting_list = []  # Initialize waiting list

    # Sort operations by operation time
    sorted_operations = sorted(operations, key=lambda i: operation_times[i], reverse=True)

    # Assign operations
    for operation in sorted_operations:
        assigned = False
        for day in days:
            for dept in [4, 7, 5, 2, 1, 6, 8, 3, 10, 9]:
                for room in [r for r in rooms if f"_{dept}" in r]:
                    if room_daily_times[room][day] >= operation_times[operation] and weights[operation][room] == 1:
                        schedule[room][day].append(operation)
                        room_daily_times[room][day] -= operation_times[operation]
                        assigned = True
                        break
                if assigned:
                    break
            if assigned:
                break
        if not assigned:
            waiting_list.append(operation)  # Add operation to waiting list

    # Try to redistribute waiting list operations
    for operation in waiting_list:
        assigned = False
        for day in days:
            for room in rooms:
                if room_daily_times[room][day] >= operation_times[operation] and weights[operation][room] == 1:
                    schedule[room][day].append(operation)
                    room_daily_times[room][day] -= operation_times[operation]
                    assigned = True
                    break
            if assigned:
                break
        if assigned:
            waiting_list.remove(operation)

    return schedule, waiting_list

schedule, waiting_list = heuristic_algorithm(operations, operation_times, weights, rooms, days, room_daily_times)

print("Schedule:")
for room in rooms:
    for day in days:
        if schedule[room][day]:  # Check if the schedule is not empty
            print(f"Room {room}, Day {day}: {schedule[room][day]}")

print(f"Waiting List: {waiting_list}")

Schedule:
Room 2_9, Day 1: [1, 298]
Room 2_9, Day 2: [65, 220, 30, 18]
Room 4_6, Day 1: [301, 260]
Room 4_6, Day 2: [152, 183, 91, 107]
Room 1_9, Day 1: [162, 195, 3]
Room 1_9, Day 2: [90, 108, 241, 50, 236]
Room R4_7, Day 1: [42, 193, 11]
Room R4_7, Day 2: [155, 112, 250, 175]
Room R4_7, Day 3: [270]
Room 6_5, Day 1: [257, 38, 110]
Room 6_5, Day 2: [111, 219, 290, 8]
Room 6_5, Day 3: [80, 6, 170]
Room 8_5, Day 1: [98, 64, 118]
Room 8_5, Day 2: [63, 89, 218, 245]
Room 7_7, Day 1: [192, 254, 66]
Room 7_7, Day 2: [88, 151, 246, 288]
Room 5_3, Day 1: [128, 258, 5]
Room 5_3, Day 2: [120, 92, 221, 49]
Room 5_3, Day 3: [229]
Room 9_4, Day 1: [127, 297, 109]
Room 9_4, Day 2: [85, 147, 145, 24, 132]
Room 8_2, Day 1: [259, 41, 209]
Room 8_2, Day 2: [234, 16, 46]
Room 8_7, Day 1: [123, 223, 117]
Room 8_7, Day 2: [286, 213, 146, 9, 51]
Room R3_4, Day 1: [67, 68, 13]
Room R3_4, Day 2: [106, 143, 210, 211, 207]
Room R3_8, Day 1: [131, 188, 293]
Room R3_8, Day 2: [171, 134, 135, 235, 200, 271]
Room 